# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer/v2?filepath=ShowAccount.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 Show My Accounts

This notebook tries to display information about all Mango margin accounts that belong to a specified account.

It fetches the data from Solana, parses it, and then prints it.


## How To Use This Page

Enter the public key of the account you want to check in the value for `ACCOUNT_TO_LOOK_UP` in the box below, between the double-quote marks. Then run the notebook by choosing 'Run > Run All Cells' from the notebook menu at the top of the page.

In [ ]:
ACCOUNT_TO_LOOK_UP = ""

In [ ]:
if __name__ == "__main__":
    import mango
    import solana.publickey as publickey

    if ACCOUNT_TO_LOOK_UP == "":
        raise Exception("No account to look up - try setting the variable ACCOUNT_TO_LOOK_UP to an account public key.")

    context = mango.Context.default()
    # print("Context:", context)

    root_account_key = publickey.PublicKey(ACCOUNT_TO_LOOK_UP)
    root_account = mango.AccountInfo.load(context, root_account_key)
    if root_account is None:
        raise Exception(f"Account '{root_account_key}' could not be found.")

    print("Account:", root_account)
    if root_account.owner != mango.SYSTEM_PROGRAM_ADDRESS:
        raise Exception(f"Account '{root_account_key}' is not a root user account.")

    scout = mango.AccountScout()
    group = mango.Group.load(context)
    scout_report = scout.verify_account_prepared_for_group(context, group, root_account_key)
    print(scout_report)

    print("Balances:")
    mango.TokenValue.report(group.fetch_balances(context, root_account_key))

    prices = group.fetch_token_prices(context)

    margin_accounts = mango.MarginAccount.load_all_for_owner(context, root_account_key, group)
    print(f"Account has {len(margin_accounts)} margin account(s).")
    for margin_account in margin_accounts:
        print("Margin account:", margin_account)
        print("Balance sheet totals", margin_account.get_balance_sheet_totals(group, prices))
